In [1]:
from transformers import set_seed, GPT2LMHeadModel, GPT2TokenizerFast
from torch.utils.data import DataLoader
from tqdm import tqdm
import datasets
import numpy as np
import scipy
import torch

set_seed(0)


In [2]:
NUM_PROC = 64
SPLIT = NUM_PROC * 16
BATCH_SIZE = 64


In [3]:
# Tokens in the middle of a sentence are prefixed with a space and thus are not equal to the tokens at the beginning of a sentence.

prefix = "This is a sad story. "
model = GPT2LMHeadModel.from_pretrained("gpt2").cuda()
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def tokenization(example):
    example["text"] = [prefix + text for text in example["text"]]
    return tokenizer(
        example["text"],
        max_length=64,  # Needs to be less than 74 or something.
        truncation=True
    )

ds_arrow = datasets.load_dataset("roneneldan/TinyStories", split=f"train[:{SPLIT}]")
ds = ds_arrow.map(tokenization, batched=True, batch_size=BATCH_SIZE, num_proc=NUM_PROC)
# Prefixes should be equal.
ds[0]["text"], ds[0]["input_ids"][:10], ds[0]["attention_mask"][:10], ds[1]["input_ids"][:10], tokenizer(prefix, return_tensors="pt"), tokenizer.convert_ids_to_tokens(ds[0]["input_ids"])[:10]


Repo card metadata block was not found. Setting CardData to empty.


('This is a sad story. One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.',
 [1212, 318, 257, 6507, 1621, 13, 1881, 1110, 11, 257],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1212, 318, 257, 6507, 1621, 13, 4874, 2402, 257, 640],
 {'input_ids': tensor([[1212,  318,  257, 6507, 1621,   13,  220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])},
 

In [4]:
dl = DataLoader(ds.with_format("torch"), batch_size=BATCH_SIZE, num_workers=0)
for batch in tqdm(dl):
    batch_cuda = batch["input_ids"].cuda()
    with torch.no_grad():
        logits = model.forward(batch_cuda)

        # Do something with logits.
        


100%|██████████| 16/16 [00:02<00:00,  6.76it/s]
